In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from scipy.cluster.hierarchy import dendrogram




from sentence_transformers import SentenceTransformer

from umap import UMAP
import plotly.express as px

C:\Users\adshafi\Anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
def import_data():
    X_train = pd.read_csv('./train/X_train.csv')
    X_test = pd.read_csv('./test/X_test.csv')
    y_train = pd.read_csv('./train/y_train.csv')
    y_test = pd.read_csv('./test/y_test.csv')
    return X_train, X_test, y_train, y_test

In [3]:
X_train, X_test, y_train, y_test = import_data()

In [4]:
X_train['c_charge_desc'].unique()

array(['arrest case no charge', 'Grand Theft in the 3rd Degree',
       'Felony Driving While Lic Suspd', 'Robbery W/Firearm', 'Battery',
       'Possession of Hydrocodone', 'Burglary With Assault/battery',
       'Possession of Cocaine', 'Possess Cannabis/20 Grams Or Less',
       'Susp Drivers Lic 1st Offense', 'Uttering Forged Credit Card',
       'Aggrav Battery w/Deadly Weapon', 'Purchase Of Cocaine',
       'Agg Battery Grt/Bod/Harm', 'Burglary Unoccupied Dwelling',
       'Aggravated Assault w/Firearm', 'Disorderly Intoxication',
       'Possession Of Amphetamine', 'Manufacture Cannabis',
       'DUI- Enhanced', 'Opert With Susp DL 2nd Offens',
       'Possession Of Alprazolam', 'Deliver Cannabis', 'Child Abuse',
       'Grand Theft (Motor Vehicle)', 'Retail Theft $300 1st Offense',
       'Driving Under The Influence', 'Leave Acc/Attend Veh/More $50',
       'Felony Petit Theft', 'Burglary Dwelling Occupied',
       'Aggravated Assault W/Dead Weap', 'Possession of Cannabis',
  

In [5]:
charges = X_train['c_charge_desc'].append(X_test['c_charge_desc']).dropna().unique()

In [6]:
charges

array(['arrest case no charge', 'Grand Theft in the 3rd Degree',
       'Felony Driving While Lic Suspd', 'Robbery W/Firearm', 'Battery',
       'Possession of Hydrocodone', 'Burglary With Assault/battery',
       'Possession of Cocaine', 'Possess Cannabis/20 Grams Or Less',
       'Susp Drivers Lic 1st Offense', 'Uttering Forged Credit Card',
       'Aggrav Battery w/Deadly Weapon', 'Purchase Of Cocaine',
       'Agg Battery Grt/Bod/Harm', 'Burglary Unoccupied Dwelling',
       'Aggravated Assault w/Firearm', 'Disorderly Intoxication',
       'Possession Of Amphetamine', 'Manufacture Cannabis',
       'DUI- Enhanced', 'Opert With Susp DL 2nd Offens',
       'Possession Of Alprazolam', 'Deliver Cannabis', 'Child Abuse',
       'Grand Theft (Motor Vehicle)', 'Retail Theft $300 1st Offense',
       'Driving Under The Influence', 'Leave Acc/Attend Veh/More $50',
       'Felony Petit Theft', 'Burglary Dwelling Occupied',
       'Aggravated Assault W/Dead Weap', 'Possession of Cannabis',
  

In [7]:
charges_clean=[i.lower()
.replace('w/o','without ')
.replace('/w','with ')
.replace('/',' ')
.replace('dl','drivers license')
.replace('driv','drivers')                
.replace('lic','license')               
.replace('priv','private')
.replace('deg','degree')
.replace('viol','violence')
.replace('veh','vehicle')
.replace('first','1st')
.replace('poss','possession')
.replace('bugl','buglary')
.strip() for i in charges]

In [8]:
robert = SentenceTransformer('stsb-roberta-large') 

In [9]:
embeddings = pd.DataFrame(np.vstack(np.array([robert.encode(i) for i in charges_clean])))

In [10]:
def cluster_plot(model):
    
    clust = model
    clust.fit(embeddings)
    
    umap = UMAP(metric='cosine',random_state=42)
    umap.fit(embeddings)
    
    data = pd.DataFrame(umap.transform(embeddings),columns=["x","y"]).assign(text=charges).assign(color=clust.labels_)
    
    fig = px.scatter(data,x='x',y='y',hover_data={'b':data['text']},color=data['color'])
    fig.update_layout(margin=dict(l=0,r=0,b=0,t=0))
    fig.update_layout(hoverlabel={'font_size':12,'bgcolor':'black'})
    fig.update_traces(marker={'size':4})
    
    return model,fig

In [11]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [15]:
model = KMeans(n_clusters=16)
kme,fig = cluster_plot(model)
fig

In [13]:
model = DBSCAN(metric='cosine',eps=.3,min_samples=5)
dbs,fig = cluster_plot(model)
fig

In [14]:
model = AgglomerativeClustering(n_clusters=20,affinity='cosine',linkage='complete')
agg,fig = cluster_plot(model)
fig

In [15]:
charge_map = {i:j for i,j in zip(charges,kme.labels_)}
charge_map['arrest case no charge']=-1
pickle.dump(charge_map, open('charge_map_v2.pkl', 'wb')) 